<center><img src="https://matematica.usm.cl/wp-content/themes/dmatUSM/assets/img/logoDMAT2.png" title="Title text" width= 800 /></center>
<hr style="height:2px;border:none"/>
<h1 align='center'> Ayudantía 7: Modelos de clasificación, librería scikit-learn</h1>

<H3 align='center'> MAT281 2023-2 </H3>

<H3 align='center'> Ayud. Alejandro Villazón G. </H3>
<hr style="height:2px;border:none"/>

Los modelos de regresión asumen que la variable de respuesta es cuantitativa, sin embargo, en muchas situaciones esta variable es cualitativa/categórica, por ejemplo el color de ojos. La idea de predecir variables categóricas es usualmente nombrada como _Clasificación_. Muchos de los problemas en los que se enfoca el Machine Learning están dentro de esta categoría, por lo mismo como vieron en clases existen una serie de algoritmos y modelos con tal de obtener los mejores resultados. 

Todos los modelos vistos en clases se encuentran en la librería `sklearn` y la forma de usarlos es muy similar, es por esto que en esta ayudantía introduciremos el algoritmo de clasificación más sencillo: _Regresión Logística_, y uno de los más intuitivos de comprender: _K Nearest Neighbours_.

Trabajaremos con el conjunto de datos `Breast Cancer`, donde el objetivo es clasificar, en bases a sus características, si un tumor es Benigno o Maligno.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_dict = datasets.load_breast_cancer()
breast_cancer = pd.DataFrame(data=data_dict.data,
                    columns=data_dict.feature_names)

breast_cancer['target'] = data_dict.target

breast_cancer.head()

`Scikit-learn` tiene su propia implementación de Regresión Logística, para un enfoque más estadístico te invito a revisar la implementación del paquete `statsmodels`. A medida que vayas leyendo e interiorizándote en la librería verás que tratan de mantener una sintaxis consistente en los distintos objetos y métodos.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X = breast_cancer.drop('target', axis=1)
y = breast_cancer['target']


Logit = LogisticRegression(
    penalty='l2',
    max_iter=10_000
)
Logit.fit(X, y)

Podemos obtener los coeficientes resultantes del ajuste a nuestros datos:

In [ ]:
Logit.intercept_

In [ ]:
Logit.coef_

Podemos predecir según el modelo entrenado:

In [ ]:
y_pred_train = Logit.predict(X)
y_pred_train[:100]

Después de entrenar el modelo, es esencial llevar a cabo una evaluación de desempeño adecuada. Sin embargo, no resulta objetivo evaluar el rendimiento del modelo utilizando los mismos datos que se utilizaron para entrenarlo. Por lo tanto, se hace necesario dividir el conjunto de datos en dos partes: el conjunto de entrenamiento (Train Set) y el conjunto de prueba (Test Set). El Test Set se utiliza exclusivamente para evaluar el rendimiento del modelo final previamente seleccionado, sin involucrarse en la selección de hiperparámetros, como la elección de la norma de penalización, que puede ser $\ell_1$, $\ell_2$, o una combinación de ambas.

Para ajustar y optimizar los hiperparámetros, se emplea un tercer conjunto llamado Validation Set, que se obtiene al separar una parte del conjunto de entrenamiento. Este conjunto se utiliza para evaluar el desempeño de diferentes modelos comparables que pertenecen a la misma categoría. Por ejemplo, si se busca comparar los siguientes modelos:
``` python 
- LogisticRegression(penalty='l2')
```
``` python
- LogisticRegression(penalty='elasticnet', l1_ratio=0.6)
```
debemos usar el Validation Set. En cambio, si quiero comparar modelos de diferentes categorías como los siguientes:
``` python 
- LogisticRegression(penalty='l1')
```
``` python 
- DecisionTreeClassifier(max_depth=5, min_samples_leaf=10)
```
debemos usar el Test Set. Es esencial conocer la diferencia entre estos conjuntos de datos, te invito a investigar por tu cuenta [[Articulo]](https://www.geeksforgeeks.org/training-vs-testing-vs-validation-sets/)[[Foro]](https://stats.stackexchange.com/questions/19048/what-is-the-difference-between-test-set-and-validation-set).

Entonces dada la discusión anterior, si queremos ver que tan bueno es nuestro modelo, podemos ajustarlo con parte de los datos (Train Set) y ver que tan bueno es (con alguna métrica) con el resto de los datos (Test Set).

La siguiente función nos permite separar los datos:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Para el proceso de [_hyperparameter tuning_](https://www.geeksforgeeks.org/hyperparameter-tuning/) podemos utilizar la función [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) de sklearn la que además nos permite realizar Validación Cruzada (lo que nos genera el Validation Set).

In [ ]:
from sklearn.model_selection import GridSearchCV

Si queremos buscar los mejores hiperparámetros de un modelo, debemos generar una o más grillas con los valores a tomar por cada hiperparámetro, para que `GridSearchCV` entrene el modelo con todas las combinaciones y luego según una métrica de evaluación entregada por nosotros nos retorne el mejor modelo.

Por ejemplo, veamos los hiperparámetros por defecto de una Regresión Logística de sklearn:

In [ ]:
LogisticRegression().get_params()

In [ ]:
param_grid = [
    # El nombre de las keys debe ser exactamente el nombre del argumento que recibe el modelo
    {'penalty': ['l1', 'l2']},
    {'penalty' : ['elasticnet'],
     'l1_ratio': [0.2, 0.5, 0.8]
    }
]

logit = LogisticRegression(solver='saga')
 
best_logit = GridSearchCV(logit, param_grid,
                          scoring = 'f1',
                          cv = 5 # Divisiones del conjunto de entrenamiento para generar conjuntos de Validación
                          )
 
best_logit.fit(X, y)

Obtengamos los resultados:

In [ ]:
best_logit.best_estimator_

In [ ]:
best_logit.best_params_

In [ ]:
best_logit.best_score_ # Promedio del score en las iteraciones del CV

También podemos recuperar la historia del entrenamiento, como el score que se obtuvo en cada conjunto de validación al realizar la CV.

In [ ]:
best_logit.cv_results_

Una vez encontrados los mejores hiperparámetros, se debe entrenar el modelo con todo el conjunto de entrenamiento. Por suerte la función `GridSearchCV` lo realiza por defecto, gracias al parámetro `refit = True`. Por lo que podemos predecir en el Test Set y evaluar el desempeño final del modelo.

In [ ]:
y_pred = best_logit.predict(X_test)

Antes de pasar a la evaluación de desempeño, es prudente recordar que la librería `sklearn` es consistente en su sintaxis, por lo que podriamos cambiar el modelo y de forma análoga podremos realizar la clasificación y sus resultados. A continuación se encuentran todos los modelos vistos en clases:

``` python
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestClassifier
```
¡¡Visita la documentación!!

## Métricas para Clasificación

Sabemos que los modelos de clasificación etiquetan a los datos a partir del entrenamiento. Por lo tanto es necesario introducir conceptos vistos en las cátedras.

Uno de ellos es la matriz de confusión. Típicamente para un clasificador binario (puede ser extendido fácilmente a un problema multiclase) se tiene:

* `TP`: Verdadero Positivo
* `FN`: Falso Negativo
* `FP`: Falso positivo
* `TN`: Verdadero Negativo

En este contexto, los valores `TP` y `TN` muestran los valores correctos que tuve al momento de realizar la predicción, mientras que los valores de de `FN` y `FP` denotan los valores en que la clasificación fue errónea.

Una manera eficaz de visualizar estos resultados es con la _matriz de confusión_

![confusion_matrix](https://miro.medium.com/max/1780/1*LQ1YMKBlbDhH9K6Ujz8QTw.jpeg)

En un principio se busca maximizar la suma de los elementos bien clasificados, sin embargo eso depende mucho del problema a resolver. Para esto se definen las siguientes métricas:

* Accuracy:

    $$\textrm{accuracy}= \frac{TP+TN}{TP+TN+FP+FN}$$
    
* Recall:

    $$\textrm{recall} = \frac{TP}{TP+FN}$$
    
* Precision:

    $$\textrm{precision} = \frac{TP}{TP+FP} $$
    
* F-score:

    $$\textrm{F-score} = 2\times \frac{  \textrm{precision} \times \textrm{recall} }{  \textrm{precision} + \textrm{recall} } $$

Estas son las más comunes, y como te imaginarás, `scikit-learn` tiene toda una artillería de selección de modelos que puedes encontrar en este [enlace](https://scikit-learn.org/stable/modules/model_evaluation.html).

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

La forma usual de visualizar el desempeño de un modelo en un problema de clasificación es a través de la matriz de confusión, para esto se presenta dos formas:

In [ ]:
# Utiliza el modelo
ConfusionMatrixDisplay.from_estimator(
    best_logit,
    X_test,
    y_test,
    cmap=plt.cm.Blues,
    display_labels=data_dict.target_names,
    text_kw={'size':15}
)
plt.title('Confusion Matrix');

In [ ]:
# Utiliza las predicciones
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred,
    cmap=plt.cm.Blues,
    display_labels=data_dict.target_names,
    text_kw={'size':15}
)
plt.title('Confusion Matrix');

El modelo no se ajusta perfectamente como es esperado (si no podríamos caer en _overfitting_), pero te puedes basar en los siguientes resultados para hacer un análisis:

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

In [ ]:
print(f"Accuracy score: {accuracy_score(y_test, y_pred):.3f}")
print(f"Recall score: {recall_score(y_test, y_pred):.3f}")
print(f"F1 score: {f1_score(y_test, y_pred):.3f}")

Incluso podemos generar un reporte mucho más rápido. [Más información](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred, target_names=data_dict.target_names))

## Curva ROC

Para finalizar te presento como graficar la curva ROC presentada en clases, esto nos da otra forma de escoger un buen clasificador.

Como casi siempre, `scikit-learn` ya tiene implementada la función y mantiene la sintaxis que usamos al generar la matriz de confusión.

In [ ]:
from sklearn.metrics import RocCurveDisplay

In [ ]:
logit_disp = RocCurveDisplay.from_estimator(
    best_logit, 
    X_test, 
    y_test,
    name = 'Logit',
    c='red'
)
plt.xlabel('Especificidad'); plt.ylabel('Sensibilidad')
plt.title('ROC Curve');

Podemos ajustar otros modelos y presentar las curvas en un mismo gráfico!

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

# Esto es solo un ejemplo!
rfc = RandomForestClassifier(n_estimators=100, max_depth=2) 
rfc.fit(X_train, y_train)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

In [ ]:
ax = plt.gca() # Get Current Axes
lda_disp = RocCurveDisplay.from_estimator(lda, X_test, y_test, ax=ax)
rfc_disp = RocCurveDisplay.from_estimator(rfc, X_test, y_test, ax=ax)
logit_disp.plot(ax=ax)

plt.xlabel('Especificidad'); plt.ylabel('Sensibilidad')
plt.title('ROC Curves');

Podemos recuperar la AUC:

In [ ]:
rfc_disp.roc_auc, logit_disp.roc_auc, lda_disp.roc_auc

Con esto llegamos al final de los contenidos, te dejo la invitación a que continues estudiando sobre modelos de aprendizaje automático y fortalezcas tus habilidades de programación, visita la página oficial de [`scikit-learn`](https://scikit-learn.org/stable/index.html), ahí podrás encontrar la documentación de los modelos y tutoriales de todo tipo! 

También te invito a que investigues temas más avanzados como redes neuronales, para eso visita la página oficial de [`keras`](https://keras.io/).

# Ejercicio